In [61]:
import os
import numpy as np
import re
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score

n_lamwin = 9
short_cutoff = 0.03
long_cutoff = 0.03
short_log_file = f'pwwp1_{n_lamwin}L_s_log.txt'
long_log_file = f'pwwp1_{n_lamwin}L_log.txt'
lig1_int = 1
lig2_int = 3

wd = r'C:\Users\scott.midgley\OneDrive - Cresset BMD Ltd\OneDrive\science\adaptive_lambda_article\calculations\fragments\matrices'
short_file = os.path.join(wd, short_log_file) 
long_file = os.path.join(wd, long_log_file) 

In [62]:
#Short matrices
count = 0
all_short_mat = []
with open(short_file, 'r') as infile:
    content = infile.readlines()
    for i, line in enumerate(content):
        if ('Free' in line):
            if ('Intermediate' not in line):
            #if line.startswith('ligand'):
                if (content[i+1].startswith('0.')) or (content[i+1].startswith('1.')):           
                    go = i
                    stop = go+n_lamwin
                    while go <= stop:
                        all_short_mat.append(content[go])
                        go += 1 

all_short_mat = ([(s.replace('\n', '')) for s in all_short_mat])
asm = np.array(all_short_mat) 

# Long matrices
count = 0
all_long_mat = []
with open(long_file, 'r') as infile:
    content = infile.readlines()
    for i, line in enumerate(content):
        if ('Bound' in line):
            if ('Intermediate' not in line):
            #if line.startswith('ligand'):
                if (content[i+1].startswith('0.')) or (content[i+1].startswith('1.')):            
                    go = i
                    stop = go+n_lamwin
                    while go <= stop:
                        all_long_mat.append(content[go])
                        go += 1 

all_long_mat = ([(s.replace('\n', '')) for s in all_long_mat])
alm = np.array(all_long_mat) 

In [ ]:
print(asm)

In [ ]:
print(alm)

In [ ]:
overall_match_count = []
short_count = []
long_count = []
for i, iline in enumerate(alm):
    if ('Bound' in iline) :
        if (alm[i+1].startswith('0.')) or (alm[i+1].startswith('1.')):
            long_mat = alm[i+1:i+(n_lamwin+1)]
            long_mat_title = alm[i]
            split_line = re.split('~| ',iline)
            l1l = split_line[lig1_int]
            l2l = split_line[lig2_int]
            #print(split_line)

            for j, jline in enumerate(asm):
                if ('Free' in jline) :
                    if (asm[j+1].startswith('0.')) or (asm[j+1].startswith('1.')):
                        short_mat = asm[j+1:j+(n_lamwin+1)]
                        short_mat_title = asm[j]
                        split_line = re.split('~| ',jline)
                        l1s = split_line[lig1_int]
                        l2s = split_line[lig2_int]

                        if (l1l == l1s) and (l2l == l2s) and ('Free' in jline):
                            print(split_line)
                            GO = 1
                            odlm = []
                            for line in long_mat:
                                split_line = line.split()
                                elem = split_line[GO]
                                odlm.append(elem)
                                GO += 1
                                if GO == n_lamwin:
                                    break

                            odlm_cat = []
                            for i in odlm:
                                if float(i) > long_cutoff:
                                    odlm_cat.append(0)
                                    long_count.append(0)
                                else:
                                    odlm_cat.append(1)
                                    long_count.append(1)
                            
                            GO = 1
                            odsm = []
                            for line in short_mat:
                                split_line = line.split()
                                elem = split_line[GO]
                                odsm.append(elem)
                                GO += 1
                                if GO == n_lamwin:
                                    break

                            #print(odfm)

                            odsm_cat = []
                            for i in odsm:
                                if float(i) > short_cutoff:
                                    odsm_cat.append(0)
                                    short_count.append(0)
                                else:
                                    odsm_cat.append(1)
                                    short_count.append(1)


In [ ]:
y_true = long_count
y_pred = short_count

# y_true = short_count
# y_pred = long_count

print(f'Total number of lambda windows not involving intermediates: {len(y_true)}')

# Confusion matrix
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
# print(f'True Positive: {tp}', 
#       f'\nTrue Negative: {tn}', 
#       f'\nFalse Positive: {fp}',
#       f'\nFalse Negative: {fn}')

# acc = (tp + tn) / (tp + tn + fp + fn)
# pre = tp / (tp + fp)
# rec = tp / (tp + fn)

# print(f'\nAccuracy Score: {round(accuracy_score(y_true, y_pred), 2)}', 
#       f'\nPrecision Score: {round(precision_score(y_true, y_pred), 2)}', 
#       f'\nRecall Score: {round(recall_score(y_true, y_pred), 2)}')

df_lst = []
df_lst.append(round(accuracy_score(y_true, y_pred), 2))
df_lst.append(round(precision_score(y_true, y_pred), 2))
df_lst.append(round(recall_score(y_true, y_pred), 2))
df_lst.append(len(y_true))
df_lst.append(tp)
df_lst.append(tn)
df_lst.append(fp)
df_lst.append(fn)


df = pd.DataFrame(df_lst).T
df.columns = ['Acc', 'Prec', 'Rec', 'TOT L', 'tp', 'tn', 'fp', 'fn']
df.to_csv('tmp.csv')

os.system("start EXCEL.EXE tmp.csv")